In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_dc_sku_list 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH supra AS (
    SELECT DISTINCT dc_id FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive WHERE supra = 'YES'
),
dc_golive AS (
    SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY dc_id ORDER BY business_unit DESC) AS rn
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive
) sub
WHERE rn = 1

),
store_sku_mdq AS (
    SELECT DISTINCT
        a.product_id,
        a.dc_id,
        CASE
            WHEN a.replenishment_mode = 'BS' THEN 'BUILD_STOCK' 
            WHEN a.replenishment_mode = 'FT' THEN 'FLOW_THROUGH' 
        END dc_mode,
        IF(c.business_unit IS NULL, NULL, "YES") winare_dc_golive,
        IF(d.dc_id IS NULL, "NO", "YES") supra,
        c.contact_name,
        c.contact_tel
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly a1 ON a.store_id = a1.store_id AND a.product_id = a1.product_id
        LEFT JOIN dc_golive c ON a.dc_id = c.dc_id
        LEFT JOIN supra d ON a.dc_id = d.dc_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dc_fresh_sku e ON a.dc_id = e.dc_id AND a.product_id = e.product_id
    WHERE a.dc_sku_status = 'ACTIVE'
        AND (
                (a1.mdq_available = "AVAILABLE" AND a.mch2_id IN ("102","201","202","203")) 
                    OR 
                e.product_id IS NOT NULL
        ) 
),
region_sku_vendor AS (
    SELECT
        region,
        product_id,
        vendor_id,
        vendor_name,
        vendor_subrange,
        uom  -- 28/09/2022: Lấy thông tin ORDER_UNIT từ đây (TAB03)
    FROM (
        SELECT
            region,
            product_id,
            vendor_id,
            vendor_name,
            vendor_subrange,
            uom,
            ROW_NUMBER() OVER (PARTITION BY region, product_id ORDER BY 1) AS rn1
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku
    ) sub
    WHERE rn1 = 1
)
,
core_range AS (
    SELECT
        dc_id,
        product_id,
        CASE
            WHEN assortment_type = 'Core 1' THEN 1
            WHEN assortment_type = 'HERO' THEN 2
            WHEN assortment_type = 'Core 2' THEN 3
            WHEN assortment_type = 'Core 3' THEN 4
            WHEN assortment_type = 'HI_BEAUTY' THEN 5
            ELSE 6
        END AS pri,
        assortment_type AS core_range
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly
    QUALIFY ROW_NUMBER() OVER (
        PARTITION BY dc_id, product_id
        ORDER BY 
            CASE
                WHEN assortment_type = 'Core 1' THEN 1
                WHEN assortment_type = 'HERO' THEN 2
                WHEN assortment_type = 'Core 2' THEN 3
                WHEN assortment_type = 'Core 3' THEN 4
                WHEN assortment_type = 'HI_BEAUTY' THEN 5
                ELSE 6
            END
    ) = 1
)
,
pcb AS (
    SELECT 
        a.dc_id, 
        a.product_id, 
        a.pcb, 
        ROW_NUMBER() OVER (
            PARTITION BY a.dc_id, a.product_id 
            ORDER BY a.pcb
        ) AS rn3
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_pcb a
    INNER JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive b 
        USING(dc_id, business_unit)
    QUALIFY rn3 = 1
)
,
block_by_store_sku AS (
    SELECT store_id, product_id, concat_ws(' - ', collect_set(reason)) AS reason
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku
    WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN `from` AND `to`
    GROUP BY 1,2
),
pir_current AS (
    SELECT DISTINCT vendor_id, product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_pir_current
    WHERE status = 'ACTIVE'
),
moq_all_site AS (
	SELECT    
		a.vendor_id,
		IFNULL(a.subrange_id, '') vendor_subrange,
		a.product_id,
		a.moq
	FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_pir_info a
	WHERE a.store_id IS NULL
),
mov_vendor AS (
    SELECT LTRIM('0', vendor_id) vendor_id, mov
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_master
),
mov_subrange AS (
    SELECT LTRIM('0', vendor) vendor_id, vendor_subrange, mov
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_subrange
),
auto_block_promotion_dc AS (
    SELECT a.dc_id, a.product_id, concat_ws('; ', collect_set(CONCAT(a.reason, ': ', a.promo_id))) reason
    FROM (
        SELECT dc_id, product_id, reason, concat_ws(',', collect_set(promo_id)) AS promo_id
        FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_dc_current
        WHERE 
            dc_id IN (SELECT DISTINCT dc_id
            FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly
            WHERE BUSINESS_UNIT = '2000')
            AND product_id NOT IN (SELECT DISTINCT product_id FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_store_sku_winare_promotion_golive)
        GROUP BY 1,2,3
    ) a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b
        USING(product_id)
    WHERE (b.manufacturer_id <> '1012' AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN '2024-12-23' AND '2025-01-31')
        OR DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) > '2025-01-31' 
    GROUP BY 1,2
)
SELECT DISTINCT
    a.dc_id,
    a.product_id,
    a.dc_mode,
    CASE
        WHEN o.product_id IS NOT NULL AND a.dc_mode = 'BUILD_STOCK' THEN 'WINARE FRESH'
        ELSE NULL
    END winare_type,
    o.order_method,
    b.region,
    c.vendor_id,
    c.vendor_name,
    c.vendor_subrange,
    IFNULL(d4.mov * 100, d3.mov) mov,
    d2.moq,
    d.order_schedule,
    IF(a.dc_mode = 'BUILD_STOCK', IF(k.core_range IN ('Core 1', 'Core 2', 'HERO') AND SIZE(SPLIT(d.order_schedule, ',')) = 1, 9, 7), IFNULL(d.dbo, CEIL(7 / NULLIF(size(split(d.order_schedule, ',')), 0)))) dbo,
    IFNULL(d.leadtime, IFNULL(d1.leadtime, 7)) + IF (DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN '2025-04-19' AND '2025-04-23', 3, 0) leadtime,
    IF(a.dc_mode = 'BUILD_STOCK',
        CASE
            WHEN k.core_range IN ('Core 1', 'HERO') THEN 3
            WHEN k.core_range = 'Core 2' THEN 2
            ELSE 1
        END,
        NULL) + IF(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN '2025-01-07' AND '2025-01-18', 2, 0) safety_stock,
    c.uom AS order_unit, -- 28/09/2022: Lấy thông tin ORDER_UNIT từ TAB03
    a.winare_dc_golive,
    a.supra,
    a.contact_name,
    a.contact_tel,
    IF(a.supra = "YES", NULL, 7) po_pending_day,
    IF(a3.store_id IS NULL AND j.product_id IS NULL AND a4.vendor_id IS NOT NULL AND a5.dc_id IS NULL, "NO", "YES") is_winare_block,
    l.capacity,
    h.manufacturer_id,
    n.priority,
    k.core_range,
    -- IFNULL(m.pcb, IF(m2.unit = 'T', IFNULL(SAFE_DIVIDE(m2.numerator, m2.denomintR), 1), 1)) pcb, -- 17/04/2024: ngắt rule default
    m.pcb,
    CONCAT(
        IF(a3.reason IS NULL, '', CONCAT(a3.reason, ', ')),
        IF(j.reason IS NULL, '', CONCAT(j.reason, ', ')),
        IF(a4.vendor_id IS NOT NULL, '', 'PIR, '),
        IF(a5.reason IS NULL, '', a5.reason)
    ) block_reason,
    CONCAT(
        IF(a3.reason IS NULL AND j.reason IS NULL, '', 'KSTK, '),
        IF(a4.vendor_id IS NOT NULL, '', 'PIR, '),
        IF(a5.reason IS NULL, '', 'PROMOTION')
    ) block_type,
    CONCAT(
        'NULL ',
        IF(b.region IS NULL, 'REGION, ', ''),
        IF(c.vendor_id IS NULL, 'VENDOR_ID, ', ''),
        IF(c.uom IS NULL, 'ORDER_UNIT, ', ''), -- 28/09/2022: Lấy thông tin ORDER_UNIT từ TAB03
        IF(a.winare_dc_golive IS NULL, 'WINARE_DC_GOLIVE, ', ''),
        IF(m.pcb IS NULL, 'PCB', '')
    ) check_null
FROM store_sku_mdq a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_region_dc b ON a.dc_id = b.dc_id
    LEFT JOIN region_sku_vendor c ON b.region = c.region AND a.product_id = c.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_order_config_vendor_dc d ON a.dc_id = d.dc_id AND c.vendor_id = d.vendor_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_leadtime_order_vendor d1 ON a.dc_id = d1.dc_id AND c.vendor_id = d1.vendor_id
    LEFT JOIN moq_all_site d2 ON c.vendor_id = d2.vendor_id AND c.vendor_subrange = d2.vendor_subrange AND a.product_id = d2.product_id
    LEFT JOIN mov_vendor d3 ON c.vendor_id = d3.vendor_id
    LEFT JOIN mov_subrange d4 ON c.vendor_id = d4.vendor_id AND c.vendor_subrange = d4.vendor_subrange
    LEFT JOIN block_by_store_sku a3 ON a.dc_id = a3.store_id AND a.product_id = a3.product_id
    LEFT JOIN pir_current a4 ON c.vendor_id = a4.vendor_id AND a.product_id = a4.product_id
    LEFT JOIN auto_block_promotion_dc a5 ON a.dc_id = a5.dc_id AND a.product_id = a5.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_sku j ON a.product_id = j.product_id
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product h ON a.product_id = h.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store i ON a.dc_id = i.store_id
    LEFT JOIN core_range k ON a.dc_id = k.dc_id AND a.product_id = k.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_capacity_by_dc_rop l ON a.dc_id = l.dc_id
    LEFT JOIN pcb m ON a.dc_id = m.dc_id AND a.product_id = m.product_id
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit WHERE unit = 'T') m2 ON a.product_id = m2.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_mnf_priority n ON a.dc_id = n.dc_id AND n.manufacturer_id = h.manufacturer_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dc_fresh_sku o ON a.product_id = o.product_id AND a.dc_id = o.dc_id
;
""")